# Oracle Database with AWS Bedrock for RAG Search

This notebook demonstrates how to integrate Oracle Database with Amazon Bedrock to build a powerful Retrieval-Augmented Generation (RAG) application. We'll leverage Oracle's vector storage capabilities with AWS Bedrock's foundation models for intelligent, context-aware search.

## Architecture Overview
- **Oracle AI Database 26ai**: Vector database for storing and searching embeddings
- **Amazon Bedrock**: Serverless foundation models for embeddings and LLM inference
- **RAG Pattern**: Ground LLM responses with relevant context from your Oracle database

> **Note**: For Google Colab, you'll need to set up AWS credentials securely and ensure connectivity to your Oracle Database@AWS instance.

## 1. Installation & Dependencies

Install required Python packages for AWS Bedrock, Oracle Database, and LangChain integration.

In [ ]:
import subprocess
import sys

def install_packages():
    """Install required packages"""
    packages = [
        'python-oracledb',
        'boto3',
        'langchain',
        'langchain-aws',
        'langchain-text-splitters',
    ]
    
    for package in packages:
        print(f"Installing {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
    
    print("✓ All packages installed successfully!")

install_packages()

## 2. Import Libraries

In [ ]:
import json
import boto3
import oracledb
import time
from typing import List, Dict
from langchain.text_splitter import RecursiveCharacterTextSplitter
import getpass

print("✓ Libraries imported successfully!")

## 3. Configuration

Configure AWS and Oracle Database credentials. For Google Colab, store credentials securely using environment variables or Google Colab secrets.

In [ ]:
# Configuration Parameters
CONFIG = {
    # AWS Configuration
    'AWS_REGION': 'us-east-1',
    'BEDROCK_EMBEDDINGS_MODEL': 'amazon.titan-embed-text-v2:0',
    'BEDROCK_LLM_MODEL': 'anthropic.claude-3-5-sonnet-20241022',
    
    # Oracle Configuration
    'ORACLE_USER': 'admin',
    'ORACLE_PASSWORD': '',  # Will be prompted
    'ORACLE_HOST': '',      # Will be prompted
    'ORACLE_PORT': 1522,
    'ORACLE_SERVICE': 'FREEPDB1',
    
    # RAG Configuration
    'CHUNK_SIZE': 1000,
    'CHUNK_OVERLAP': 200,
    'TOP_K': 3,
    'EMBEDDING_DIMENSION': 1536,
}

# For Google Colab, prompt for sensitive information
print("AWS and Oracle Configuration Setup")
print("=" * 50)

# AWS credentials should be set via environment variables or Colab secrets
# CONFIG['AWS_ACCESS_KEY_ID'] = getpass.getpass("Enter AWS Access Key ID: ")
# CONFIG['AWS_SECRET_ACCESS_KEY'] = getpass.getpass("Enter AWS Secret Access Key: ")

# Oracle Database credentials
CONFIG['ORACLE_HOST'] = input("Enter Oracle Database Host: ")
CONFIG['ORACLE_PASSWORD'] = getpass.getpass("Enter Oracle Database Password: ")

print("✓ Configuration loaded!")

## 4. Initialize AWS and Oracle Connections

In [ ]:
# Initialize Bedrock client
bedrock_client = boto3.client(
    'bedrock-runtime',
    region_name=CONFIG['AWS_REGION']
)

# Test Bedrock connectivity
try:
    bedrock_client.list_foundation_models()
    print("✓ Bedrock connection successful!")
except Exception as e:
    print(f"✗ Bedrock connection failed: {e}")
    print("Make sure AWS credentials are set up properly via environment variables or boto3 configuration")

# Initialize Oracle connection
try:
    oracle_connection = oracledb.connect(
        user=CONFIG['ORACLE_USER'],
        password=CONFIG['ORACLE_PASSWORD'],
        dsn=f"{CONFIG['ORACLE_HOST']}:{CONFIG['ORACLE_PORT']}/{CONFIG['ORACLE_SERVICE']}"
    )
    print("✓ Oracle Database connection successful!")
except Exception as e:
    print(f"✗ Oracle connection failed: {e}")
    print("Please check your database credentials and network connectivity")

## 5. Create Database Tables

Create tables for vectors and RAG session history in Oracle.

In [ ]:
def create_tables():
    """Create necessary tables for RAG application"""
    cursor = oracle_connection.cursor()
    
    # Create documents table
    create_docs_table = """
    CREATE TABLE IF NOT EXISTS documents (
        doc_id VARCHAR2(256) PRIMARY KEY,
        content CLOB,
        embedding VECTOR(1536),
        metadata JSON,
        created_at TIMESTAMP DEFAULT SYSTIMESTAMP
    )
    """
    
    # Create vector index
    create_index = """
    CREATE INDEX IF NOT EXISTS doc_vector_idx ON documents(embedding) 
        INDEXTYPE IS VECTOR_INDEX
    """
    
    # Create RAG sessions table
    create_sessions_table = """
    CREATE TABLE IF NOT EXISTS rag_sessions (
        session_id VARCHAR2(256) PRIMARY KEY,
        query VARCHAR2(4000),
        response CLOB,
        context CLOB,
        model_id VARCHAR2(256),
        created_at TIMESTAMP DEFAULT SYSTIMESTAMP
    )
    """
    
    try:
        cursor.execute("DROP TABLE documents")
        oracle_connection.commit()
    except:
        pass
    
    try:
        cursor.execute("DROP TABLE rag_sessions")
        oracle_connection.commit()
    except:
        pass
    
    # Create tables
    cursor.execute(create_docs_table)
    oracle_connection.commit()
    print("✓ documents table created")
    
    cursor.execute(create_sessions_table)
    oracle_connection.commit()
    print("✓ rag_sessions table created")
    
    cursor.close()

# Create tables
create_tables()
print("✓ Database tables initialized!")

## 6. Embedding Generation

Create embeddings using Amazon Bedrock's Titan Embeddings model.

In [ ]:
def generate_embedding(text: str) -> List[float]:
    """
    Generate embedding using Amazon Titan Embeddings model
    
    Args:
        text: Input text to embed
        
    Returns:
        List of embedding values
    """
    response = bedrock_client.invoke_model(
        modelId=CONFIG['BEDROCK_EMBEDDINGS_MODEL'],
        contentType='application/json',
        accept='application/json',
        body=json.dumps({
            "inputText": text,
            "dimensions": CONFIG['EMBEDDING_DIMENSION'],
            "normalize": True
        })
    )
    
    embedding = json.loads(response['body'].read())['embedding']
    return embedding

# Test embedding generation
test_text = "Oracle Database is integrated with AWS Bedrock for RAG search"
test_embedding = generate_embedding(test_text)
print(f"✓ Generated embedding with {len(test_embedding)} dimensions")
print(f"  Sample values: {test_embedding[:5]}")

## 7. Document Ingestion

Load documents, split into chunks, and store embeddings in Oracle.

In [ ]:
def ingest_documents(text: str, source: str = "sample"):
    """
    Ingest documents into Oracle vector store
    
    Args:
        text: Document content to ingest
        source: Source identifier for the documents
    """
    # Split into chunks
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=CONFIG['CHUNK_SIZE'],
        chunk_overlap=CONFIG['CHUNK_OVERLAP']
    )
    chunks = splitter.split_text(text)
    
    # Generate embeddings and store
    cursor = oracle_connection.cursor()
    
    for i, chunk in enumerate(chunks):
        print(f"Processing chunk {i+1}/{len(chunks)}...", end='\r')
        
        embedding = generate_embedding(chunk)
        doc_id = f"{source}_chunk_{i}_{int(time.time())}"
        
        cursor.execute(
            """
            INSERT INTO documents (doc_id, content, embedding, metadata)
            VALUES (:1, :2, :3, :4)
            """,
            [
                doc_id,
                chunk,
                embedding,
                json.dumps({"chunk_index": i, "source": source})
            ]
        )
    
    oracle_connection.commit()
    cursor.close()
    
    print(f"\n✓ Ingested {len(chunks)} document chunks from {source}")
    return len(chunks)

# Example: Ingest sample documents
sample_documents = """
Oracle AI Database 26ai is the newest version of Oracle's enterprise database with built-in AI capabilities.
It provides native vector search functionality for machine learning applications.
Integration with AWS Bedrock allows seamless connection to foundation models.
Amazon Bedrock provides access to foundation models through a unified API.
You can use Bedrock for embeddings generation and language model inference.
RAG (Retrieval-Augmented Generation) improves LLM responses by grounding them with relevant context.
Oracle Vector Index enables efficient similarity search for vector embeddings.
The combination of Oracle and Bedrock creates a powerful knowledge base system.
"""

chunks_ingested = ingest_documents(sample_documents, source="tutorial")

## 8. Retrieval: Vector Similarity Search

Retrieve relevant documents from Oracle using vector similarity search.

In [ ]:
def retrieve_context(query: str, top_k: int = None) -> List[Dict]:
    """
    Retrieve relevant documents from Oracle vector store
    
    Args:
        query: User query string
        top_k: Number of top results to return
        
    Returns:
        List of relevant documents with similarity scores
    """
    if top_k is None:
        top_k = CONFIG['TOP_K']
    
    # Generate query embedding
    query_embedding = generate_embedding(query)
    
    # Perform similarity search in Oracle
    cursor = oracle_connection.cursor()
    cursor.execute(
        """
        SELECT doc_id, content, VECTOR_DISTANCE(embedding, :1) AS distance
        FROM documents
        ORDER BY distance
        FETCH FIRST :2 ROWS ONLY
        """,
        [query_embedding, top_k]
    )
    
    results = cursor.fetchall()
    cursor.close()
    
    context = [
        {
            "id": r[0],
            "content": r[1],
            "score": float(r[2])
        }
        for r in results
    ]
    
    return context

# Test retrieval
test_query = "How to integrate Oracle with AWS Bedrock for RAG?"
retrieved_docs = retrieve_context(test_query)

print(f"Query: {test_query}")
print(f"\nRetrieved {len(retrieved_docs)} documents:\n")
for i, doc in enumerate(retrieved_docs, 1):
    print(f"Document {i}: {doc['id']}")
    print(f"Similarity Score: {doc['score']:.4f}")
    print(f"Content: {doc['content'][:150]}...\n")

## 9. LLM Response Generation

Generate responses using Bedrock's Claude model, grounded with retrieved context.

In [ ]:
def generate_rag_response(query: str, context_docs: List[Dict]) -> str:
    """
    Generate response using Bedrock LLM with context
    
    Args:
        query: User query
        context_docs: List of context documents from retrieval
        
    Returns:
        Generated response from the LLM
    """
    # Prepare context string
    context_str = "\n\n".join(
        [f"[{doc['id']}]\n{doc['content']}" for doc in context_docs]
    )
    
    # Prepare system prompt
    system_prompt = """You are a helpful assistant that answers questions based on the provided context. 
Always reference the source document when applicable.
If the context doesn't contain relevant information, say so clearly."""
    
    user_message = f"""Context:
{context_str}

Question: {query}

Please provide a comprehensive answer based on the context above."""
    
    # Invoke Bedrock LLM
    response = bedrock_client.invoke_model(
        modelId=CONFIG['BEDROCK_LLM_MODEL'],
        contentType='application/json',
        accept='application/json',
        body=json.dumps({
            "anthropic_version": "bedrock-2023-06-01",
            "max_tokens": 1024,
            "system": system_prompt,
            "messages": [
                {
                    "role": "user",
                    "content": user_message
                }
            ]
        })
    )
    
    response_text = json.loads(response['body'].read())['content'][0]['text']
    return response_text

# Test LLM response generation
test_query = "What are the key benefits of Oracle AI Database?"
test_context = retrieve_context(test_query)

if test_context:
    response = generate_rag_response(test_query, test_context)
    print(f"Query: {test_query}\n")
    print(f"Response:\n{response}")
else:
    print("No context retrieved for the query")

## 10. Complete RAG Pipeline

Integrate all components into a complete RAG pipeline with session tracking.

In [ ]:
def rag_pipeline(query: str, top_k: int = None) -> Dict:
    """
    Complete RAG pipeline: retrieve context and generate response
    
    Args:
        query: User query
        top_k: Number of top documents to retrieve
        
    Returns:
        Dictionary with query, response, context, and session info
    """
    if top_k is None:
        top_k = CONFIG['TOP_K']
    
    # Retrieve context
    context_docs = retrieve_context(query, top_k)
    
    if not context_docs:
        return {
            "query": query,
            "response": "No relevant context found in the knowledge base.",
            "context": [],
            "session_id": None
        }
    
    # Generate response
    response = generate_rag_response(query, context_docs)
    
    # Store in session history
    session_id = f"session_{int(time.time())}"
    cursor = oracle_connection.cursor()
    
    cursor.execute(
        """
        INSERT INTO rag_sessions 
        (session_id, query, response, context, model_id)
        VALUES (:1, :2, :3, :4, :5)
        """,
        [
            session_id,
            query,
            response,
            json.dumps([{"id": d["id"], "score": d["score"]} for d in context_docs]),
            CONFIG['BEDROCK_LLM_MODEL']
        ]
    )
    oracle_connection.commit()
    cursor.close()
    
    return {
        "session_id": session_id,
        "query": query,
        "response": response,
        "context": context_docs
    }

# Test the complete RAG pipeline
print("=" * 60)
print("Testing Complete RAG Pipeline")
print("=" * 60)

test_queries = [
    "What is Oracle AI Database?",
    "How does Bedrock help with AI applications?",
    "What is RAG and why is it useful?"
]

results = []
for query in test_queries:
    print(f"\nQuery: {query}")
    result = rag_pipeline(query)
    results.append(result)
    print(f"Response: {result['response'][:200]}...")
    print(f"Session ID: {result['session_id']}")

print("\n✓ RAG Pipeline tests completed!")

## 11. Utility Functions

Helper functions for interactive queries, diagnostics, and cleanup.

In [ ]:
def get_database_stats() -> Dict:
    """Get statistics about stored documents"""
    cursor = oracle_connection.cursor()
    
    cursor.execute("SELECT COUNT(*) FROM documents")
    doc_count = cursor.fetchone()[0]
    
    cursor.execute("SELECT COUNT(*) FROM rag_sessions")
    session_count = cursor.fetchone()[0]
    
    cursor.close()
    
    return {
        "total_documents": doc_count,
        "total_sessions": session_count
    }

def display_session_history(limit: int = 5):
    """Display recent RAG sessions"""
    cursor = oracle_connection.cursor()
    
    cursor.execute(
        """
        SELECT session_id, query, created_at 
        FROM rag_sessions 
        ORDER BY created_at DESC 
        FETCH FIRST :1 ROWS ONLY
        """,
        [limit]
    )
    
    print(f"Recent Sessions (last {limit}):\n")
    for row in cursor.fetchall():
        print(f"Session: {row[0]}")
        print(f"Query: {row[1]}")
        print(f"Time: {row[2]}\n")
    
    cursor.close()

def check_connectivity():
    """Verify connections to AWS and Oracle"""
    print("Connection Status:")
    print("-" * 40)
    
    # Check Bedrock
    try:
        bedrock_client.list_foundation_models()
        print("✓ Bedrock: Connected")
    except Exception as e:
        print(f"✗ Bedrock: Failed - {e}")
    
    # Check Oracle
    try:
        cursor = oracle_connection.cursor()
        cursor.execute("SELECT 1 FROM DUAL")
        print("✓ Oracle: Connected")
        cursor.close()
    except Exception as e:
        print(f"✗ Oracle: Failed - {e}")

# Display initial stats
stats = get_database_stats()
print("Database Statistics:")
print(f"  Documents: {stats['total_documents']}")
print(f"  Sessions: {stats['total_sessions']}")
print()

# Check connectivity
check_connectivity()

## 12. Interactive RAG Query

Ask custom questions and get RAG-powered responses.

In [ ]:
# Example: Interactive query
# Uncomment and modify for your custom query

custom_query = "What are the best practices for RAG systems?"

print("=" * 60)
print("Running RAG Query")
print("=" * 60)
print(f"\nQuery: {custom_query}\n")

result = rag_pipeline(custom_query)

print("Response:")
print("-" * 60)
print(result['response'])
print("-" * 60)

print(f"\nContext Documents ({len(result['context'])}):")
for i, doc in enumerate(result['context'], 1):
    print(f"\n{i}. {doc['id']}")
    print(f"   Similarity Score: {doc['score']:.4f}")
    print(f"   Preview: {doc['content'][:100]}...")

print(f"\nSession ID: {result['session_id']}")

## 13. Best Practices & Troubleshooting

### Vector Quality Tips
- Use consistent embedding model throughout pipeline
- Normalize embeddings for accurate similarity search
- Test embedding quality with known good vs. bad examples

### Chunk Management
- Optimal chunk size: 512-1000 tokens
- Overlap chunks by 20-30% for context continuity
- Store chunk metadata for result filtering

### Performance Optimization
- Create appropriate indexes on vector columns
- Use batch operations for large document sets
- Cache frequently accessed embeddings
- Monitor Bedrock token usage for cost optimization

### Security Best Practices
- Use IAM roles instead of access keys
- Encrypt Oracle database connections (SSL/TLS)
- Implement VPC endpoints for Bedrock access
- Audit and log all RAG queries

### Troubleshooting Common Issues

In [ ]:
def troubleshoot_connectivity():
    """Diagnose connection issues"""
    print("Troubleshooting: Connection Tests")
    print("=" * 60)
    
    # Test database
    print("\n1. Testing Oracle Database Connection...")
    try:
        cursor = oracle_connection.cursor()
        cursor.execute("SELECT 1 FROM DUAL")
        result = cursor.fetchone()
        cursor.close()
        print("   ✓ Oracle database connection successful")
    except Exception as e:
        print(f"   ✗ Oracle database error: {e}")
        print("   Check: hostname, port, credentials, network connectivity")
    
    # Test Bedrock
    print("\n2. Testing Bedrock Connection...")
    try:
        bedrock_client.list_foundation_models()
        print("   ✓ Bedrock connection successful")
    except Exception as e:
        print(f"   ✗ Bedrock error: {e}")
        print("   Check: AWS credentials, region, IAM permissions")
    
    # Test embedding
    print("\n3. Testing Embedding Generation...")
    try:
        test_embed = generate_embedding("test")
        print(f"   ✓ Embedding successful ({len(test_embed)} dimensions)")
    except Exception as e:
        print(f"   ✗ Embedding error: {e}")
        print("   Check: Bedrock model availability, IAM permissions")

# Run troubleshooting
# troubleshoot_connectivity()

def check_embedding_dimensions():
    """Verify embedding dimensions match vector column"""
    embedding = generate_embedding("test")
    dim = len(embedding)
    expected_dim = CONFIG['EMBEDDING_DIMENSION']
    
    print(f"Embedding Dimensions Check:")
    print(f"  Actual: {dim}")
    print(f"  Expected: {expected_dim}")
    
    if dim == expected_dim:
        print("  ✓ Dimensions match")
    else:
        print("  ✗ Dimension mismatch! Update vector column definition")
        return False
    
    return True

# check_embedding_dimensions()

## 14. Cleanup & Summary

### Summary
You have successfully built a complete RAG system that:
- ✓ Integrates Oracle Database with AWS Bedrock
- ✓ Generates and stores vector embeddings
- ✓ Performs similarity search on document vectors
- ✓ Grounds LLM responses with retrieved context
- ✓ Tracks RAG sessions and conversation history

### Next Steps
1. **Add More Documents**: Ingest your own documents using `ingest_documents()`
2. **Deploy to Production**: Use EC2/Lambda with Auto Scaling
3. **Build UI**: Create Streamlit or FastAPI application
4. **Monitor**: Set up CloudWatch endpoints and alarms
5. **Optimize**: Implement caching and load balancing for high volume

### Resources
- [Oracle AI Database 26ai Documentation](https://docs.oracle.com/en/database/oracle/oracle-database/26/)
- [Amazon Bedrock User Guide](https://docs.aws.amazon.com/bedrock/latest/userguide/)
- [Oracle Vector Database](https://docs.oracle.com/en/database/oracle/oracle-database/26/arpls/DBMS_VECTOR.html)

In [ ]:
def cleanup():
    """Clean up resources"""
    print("Cleaning up resources...")
    
    try:
        if oracle_connection:
            oracle_connection.close()
            print("✓ Oracle connection closed")
    except Exception as e:
        print(f"✗ Error closing connection: {e}")

# Display final statistics
print("\n" + "=" * 60)
print("RAG System Status")
print("=" * 60)

stats = get_database_stats()
print(f"\nTotal Documents Stored: {stats['total_documents']}")
print(f"Total Sessions: {stats['total_sessions']}")

print("\n✓ RAG System Ready!")
print("\nTo perform a RAG query, use: rag_pipeline(your_question)")
print("To ingest more documents, use: ingest_documents(text, source='name')")
print("To cleanup resources, call: cleanup()")

# Note: Do not call cleanup() unless you want to end the session
# cleanup()